In [ ]:
%pip install git+https://github.com/Kajachuan/ceia-final-project

  Cloning https://github.com/Kajachuan/ceia-final-project to /tmp/pip-req-build-8u9d8wdh
  Running command git clone --filter=blob:none --quiet https://github.com/Kajachuan/ceia-final-project /tmp/pip-req-build-8u9d8wdh
  Resolved https://github.com/Kajachuan/ceia-final-project to commit 234134a888ffbec9b7cafa61912276f5982a9d71
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 96.0 MB/s eta 0:

In [ ]:
%load_ext tensorboard

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

from ceia_final_project.datasets import ArgentinaSentinel2Dataset
from ceia_final_project.transforms import SegmentationTransform
from ceia_final_project.modules import LightningSegmentation

from google.colab import drive

from glob import glob

from torch.utils.data import DataLoader

from torchmetrics import MetricCollection
from torchmetrics.segmentation import DiceScore
from torchmetrics.classification import BinaryAccuracy, BinaryF1Score, BinaryPrecision, BinaryRecall, BinaryJaccardIndex

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_root_path = 'drive/MyDrive/CEIA/Trabajo Final/Dataset'

In [ ]:
test_transform = SegmentationTransform(subset='test')

In [ ]:
test_dataset = ArgentinaSentinel2Dataset(dataset_root_path, 'test', test_transform, 256)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
root_dir = 'drive/MyDrive/CEIA/Trabajo Final/Experimentos'
log_dir = f'{root_dir}/logs'
checks_dir = f'{root_dir}/checks'

In [ ]:
metrics = MetricCollection({
    "accuracy": BinaryAccuracy(),
    "precision": BinaryPrecision(),
    "recall": BinaryRecall(),
    "f1": BinaryF1Score(),
    "iou": BinaryJaccardIndex()
})

In [ ]:
for checkpoint in glob(f'{checks_dir}/**/*.ckpt', recursive=True):
  model = LightningSegmentation.load_from_checkpoint(checkpoint).model
  model.eval()

  metrics.reset()

  for x_batch, y_batch in test_loader:
    x_batch = x_batch.to(device).float()
    y_batch = y_batch.squeeze().long()

    with torch.no_grad():
      logits = model(x_batch).cpu()

    metrics.update(logits.squeeze(), y_batch)

  results = metrics.compute()
  model_name = checkpoint.split('/')[-2]

  print(f'Model: {model_name}')
  for name, value in results.items():
    print(f'{name.capitalize()}: {value:.4f}')
  print()

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 272MB/s]


Model: unet_resnet50_dice
Accuracy: 0.8174
F1: 0.8825
Iou: 0.7897
Precision: 0.8295
Recall: 0.9427



Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 111MB/s] 


Model: unet_efficientnet-b0_dice
Accuracy: 0.8155
F1: 0.8817
Iou: 0.7885
Precision: 0.8259
Recall: 0.9456

Model: unet_resnet50_focal
Accuracy: 0.8090
F1: 0.8732
Iou: 0.7749
Precision: 0.8447
Recall: 0.9035

Model: unetplusplus_resnet50_dice
Accuracy: 0.8162
F1: 0.8813
Iou: 0.7878
Precision: 0.8313
Recall: 0.9376

Model: deeplabv3plus_resnet50_dice
Accuracy: 0.8113
F1: 0.8791
Iou: 0.7843
Precision: 0.8234
Recall: 0.9430

Model: fpn_resnet50_dice
Accuracy: 0.8116
F1: 0.8781
Iou: 0.7828
Precision: 0.8294
Recall: 0.9330

Model: segformer_resnet50_dice
Accuracy: 0.7887
F1: 0.8693
Iou: 0.7688
Precision: 0.7903
Recall: 0.9658

Model: pspnet_resnet50_dice
Accuracy: 0.8011
F1: 0.8731
Iou: 0.7747
Precision: 0.8148
Recall: 0.9403



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/conv.py:454: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1031.)
  return F.conv2d(input, weight, bias, self.stride,


...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

Model: unet_tu-swinv2_base_window8_256_dice
Accuracy: 0.8132
F1: 0.8797
Iou: 0.7852
Precision: 0.8275
Recall: 0.9388

Model: ahnet__dice
Accuracy: 0.8019
F1: 0.8738
Iou: 0.7759
Precision: 0.8144
Recall: 0.9426



/usr/local/lib/python3.11/dist-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)


Model: swinunetr__dice
Accuracy: 0.8011
F1: 0.8725
Iou: 0.7738
Precision: 0.8175
Recall: 0.9353

Model: swinunetr_v2__dice
Accuracy: 0.8025
F1: 0.8739
Iou: 0.7760
Precision: 0.8162
Recall: 0.9404

Model: segresnet__dice
Accuracy: 0.8071
F1: 0.8761
Iou: 0.7795
Precision: 0.8226
Recall: 0.9369



In [ ]:
# %tensorboard --logdir "$log_dir"